# Particle Swarm Optimisation

This notebook showcases how to use the built-in Particle Swarm Optimisation (PSO) algorithm.

In [31]:
using EvoLP
using Statistics
using OrderedCollections

For this example, we will use the Michalewicz function:

In [32]:
@doc michalewicz

```
michalewicz(x; m=10)
```

The **Michalewicz** function is a `d`-dimensional function with several steep valleys, where `m` controls the steepness. `m` is usually set at 10. For 2 dimensions, `x^* = [2.20, 1.57]`, with `f(x^*) = -1.8011`.

``
f(x) = -\sum_{i=1}^{d}\sin(x_i) \sin^{2m}\left(\frac{ix_i^2}{\pi}\right)
``


In this case we will use `d=2` and `m=10`, which is the default value implemented in EvoLP.

In PSO, we use _particles_. Each particle has a position and a velocity, and remembers the best position it has visited.

We can create a population of particles in multiple ways, but EvoLP provides 2 particle generators with random positions: either uniform or following a normal distribution.

Let's use the normal generator:

In [33]:
@doc normal_rand_particle_pop

```
normal_rand_particle_pop(n, μ, Σ; rng=Random.GLOBAL_RNG)
```

Generate a population of `n` [`Particle`](@ref) using a normal distribution with means `μ``and covariance`Σ`.

`μ` expects a vector of length *l* (i.e. number of dimensions) while `Σ` expects an *l x l* matrix of covariances.

# Examples

```julia
julia> normal_rand_particle_pop(3, [-1, -1], [1 0; 0 1])
3-element Vector{Particle}:
 Particle([-2.3026589618390214, 0.25907687184121864], [0.0, 0.0], [-2.3026589618390214, 0.25907687184121864])
 Particle([-0.5118786279984703, -0.5948648935657292], [0.0, 0.0], [-0.5118786279984703, -0.5948648935657292])
 Particle([-1.3230210847731094, -1.6234307114658497], [0.0, 0.0], [-1.3230210847731094, -1.6234307114658497])
```


In [34]:
population = normal_rand_particle_pop(50, [0, 0], [1 0; 0 1])
first(population, 3)

3-element Vector{Particle}:
 Particle([0.03927725570611182, -1.5409672086890238], [0.0, 0.0], [0.03927725570611182, -1.5409672086890238])
 Particle([0.7194563875235808, 0.41669123659493], [0.0, 0.0], [0.7194563875235808, 0.41669123659493])
 Particle([0.941528800148215, -1.0003522646667797], [0.0, 0.0], [0.941528800148215, -1.0003522646667797])

Let's use the `Logbook` to save information about each iteration of the run:

In [35]:
@doc Logbook

```
Logbook(S::LittleDict)
```

A log for statistics intended for use on every iteration of an algorithm. The logbook is constructed from a `LittleDict` ordered dictionary which maps stat names (strings) to callables, such that *statname* `i` can be computed from *callable* `i`.

The resulting `Logbook` contains:

  * `S::LittleDict`: The ordered dict of stat names and callables
  * `records::AbstractVector`: A vector of NamedTuples where each field is a statistic.


In [36]:
statnames = ["avg_fit", "median_fit", "best_fit"]
callables = [mean, median, minimum]

thedict = LittleDict(statnames, callables)
logbook = Logbook(thedict)

Logbook(LittleDict{AbstractString, Function, Vector{AbstractString}, Vector{Function}}("avg_fit" => Statistics.mean, "median_fit" => Statistics.median, "best_fit" => minimum), NamedTuple{(:avg_fit, :median_fit, :best_fit)}[])

We can now use the built-in algorithm:

In [37]:
@doc PSO

```
PSO(f, population, k_max; w=1, c1=1, c2=1)
PSO(logger::Logbook, f, population, k_max; w=1, c1=1, c2=1)
```

## Arguments

  * `f::Function`: Objective function to **minimise**.
  * `population::Vector{Particle}`: a list of [`Particle`](@ref) individuals.
  * `k_max::Integer`: number of iterations.

## Keywords

  * `w`: inertia weight. Optional, by default 1.
  * `c1`: cognitive coefficient (own's position). Optional, by default 1.
  * `c2`: social coefficient (others' position). Optional, by default 1.

Returns a [`Result`](@ref).


In [38]:
results = PSO(logbook, michalewicz, population, 30);

The output was suppressed so that we can analyse each part of the result separately:

In [39]:
@show optimum(results)

@show optimizer(results)

@show iterations(results)
@show f_calls(results);

optimum(results) = -1.8012777065081398
optimizer(results) = Particle([2.1941369557532604, 1.577623088106274], [7.589188430026585e-5, 0.027062528403300177], [2.2031120352369147, 1.5715819746382012])
iterations(results) = 30
f_calls(results) = 4601


We can also take a look at the logbook's records and see how the statistics we calculated changed throughout the run:

In [40]:
for (i, I) in enumerate(logbook.records)
    print("it: $(i) with best_pos: $(I[3]) and avg_pos: $(I[1]) \n")
end

it: 1 with best_pos: -0.8532613383506504 and avg_pos: 0.03473516241252558 
it: 2 with best_pos: -0.9927048695636607 and avg_pos: -0.1577251683580324 
it: 3 with best_pos: -0.9999295278808307 and avg_pos: -0.3167917408663068 
it: 4 with best_pos: -1.253320908614812 and avg_pos: -0.2870204986951112 
it: 5 with best_pos: -1.763451683540858 and avg_pos: -0.46763573875126246 
it: 6 with best_pos: -1.7031359749076254 and avg_pos: -0.5481064225106334 
it: 7 with best_pos: -1.663800749730735 and avg_pos: -0.5040449289429634 
it: 8 with best_pos: -1.6025853335285132 and avg_pos: -0.5401496516608956 
it: 9 with best_pos: -1.7999530006299724 and avg_pos: -0.6114323104205884 
it: 10 with best_pos: -1.7896384145117228 and avg_pos: -0.545188966942549 
it: 11 with best_pos: -1.716128254760954 and avg_pos: -0.6620877763680777 
it: 12 with best_pos: -1.7853784586218633 and avg_pos: -0.733565723960469 
it: 13 with best_pos: -1.7937551111658312 and avg_pos: -0.5999299680270959 
it: 14 with best_pos: -1.7